In [ ]:
import os, pandas, numpy, moment, xmlmanip, csv, datetime, mylittlehelpers, time
from dateutil import relativedelta
os.environ['INTERFACE_CONF_FILE'] = '/home/jjorissen/interface_secrets.conf'
from tangier_api.async_api import AsyncScheduleConnection, HandleResponses
from tangier_api.api import ScheduleConnection

In [ ]:
site_id_file = '/home/jjorissen/Documents/TangierDocs/LocationSiteIds.xlsx'
site_ids = sorted(pandas.read_excel(site_id_file).dropna()['Site ID'].unique())

In [ ]:
def x_days_prior(today, days_ago):
    return today - relativedelta.relativedelta(days=days_ago)

def per_line_print(item, per_line, index):
    if index+1%per_line  == 0:
        print(item)
    else:
        print(item, end=' ')

In [ ]:
date_combos, request_list, redo_list = [], [], []
today = datetime.datetime.now()
result_df = pandas.DataFrame()
for i in range(-300, 0, 1):
    date_combos.append((x_days_prior(today, -i+1).strftime("%Y-%m-%d"), 
                        x_days_prior(today, -i).strftime("%Y-%m-%d")))

start = time.time()
for i, date_combo in enumerate(date_combos):
    request_list, good_entries, audit_list = [], [], []
    for site_id in site_ids[:10]:
        request_list.append((date_combo[0], date_combo[1], site_id))
    schedule = AsyncScheduleConnection()
    schedule_responses = schedule.get_schedules(request_list)
    good_responses, bad_responses = schedule.good_responses, schedule.bad_responses
    redo_list.extend(HandleResponses.bad_to_new_request(bad_responses))
    good_response_list = list(map(lambda x: xmlmanip.XMLSchema(x['item'].encode('utf-8')).search(siteid__contains=''), 
                                  good_responses))
    for response in good_response_list:
        if response:
            response_as_values_list = list(map(lambda x: {'start_date': date_combo[0],
                            'end_date': date_combo[1],
                            **x}, response))
            good_entries.extend(response_as_values_list)
    good_df = pandas.DataFrame(good_entries)
    result_df = result_df.append(good_df)
    per_line_print(f'{i} {mylittlehelpers.time_elapsed(start)}: {date_combo}', 4, i)

In [ ]:
schedule_df = df
provider_ids = list(schedule_df['empid'].unique())
provider_ids.remove(numpy.nan)
provider = ProviderConnection()
provider_df = pandas.DataFrame(provider.provider_info_values_list(provider_ids))
provider_df_columns = list(provider_df.columns)
provider_df_columns.remove('@action')
provider_df = provider_df[provider_df_columns]

In [ ]:
location = LocationConnection()
location_df = pandas.DataFrame(location.location_info_values_list(site_ids=site_ids))
location_df_columns = list(location_df.columns)
location_df_columns.remove('@action')
location_df = location_df[location_df_columns]

In [ ]:
schedule_df.to_csv('schedule_flat.csv', quoting=csv.QUOTE_ALL, index=False)
provider_df.to_csv('provider_flat.csv', quoting=csv.QUOTE_ALL, index=False)
location_df.to_csv('location_flat.csv', quoting=csv.QUOTE_ALL, index=False)

In [ ]:
good_response = pandas.read_pickle('good_responses')